# Databricks Auto Loader による新規データ取り込み

このノートブックでは、Databricks Auto Loaderを使用してVolumeに配置された新規CSVファイルを自動的にトランザクションテーブルに取り込みます。

## 主な機能
- **Auto Loader**: 新規ファイルの自動検出・取り込み
- **Schema定義**: CSVファイルの構造を事前定義
- **データ品質管理**: NULL値の除外処理

## 処理フロー
1. Volume内の新規CSVファイルを監視
2. 定義されたスキーマでデータを読み込み
3. NULL値を除外してデータをクリーンアップ
4. Deltaテーブルに追加保存


## Auto Loaderによるデータ取り込み処理

Volume内の新規CSVファイルを監視し、自動的にトランザクションテーブルに取り込みます。


In [0]:
# Auto Loaderを使用してVolumeからCSVファイルを読み込み
df = (
  spark.readStream                      # ストリーミング読み込みを開始
  .format("cloudFiles")                 # Auto Loaderフォーマットを指定
  .option("cloudFiles.format", "csv")   # ソースファイル形式をCSVに設定
  .option("header", "true")             # ヘッダー行ありと指定
  # CSVファイルのスキーマを事前定義（データ型を明確化）
  .schema("transaction_id INT, customer_id INT, product_id INT, store_id INT, employee_id INT, quantity INT, purchase_date DATE, _rescued_data STRING")
  .load("/Volumes/users/yukiteru_koide/volume_yukiteru_mart_transaction")  # Volumeパスを指定
  .filter("transaction_id IS NOT NULL") # データ品質管理：transaction_idがNULLのレコードを除外
)

# Deltaテーブルに書き込み
df.writeStream.format("delta") \
  .option("checkpointLocation", "/Volumes/users/yukiteru_koide/volume_yukiteru_mart_transaction") \  # チェックポイント保存場所
  .trigger(once=True) \                 # 一度だけ実行（バッチ処理）
  .toTable("users.yukiteru_koide.transactions_table")  # 対象Deltaテーブル


## 処理完了

Auto Loaderによる新規データの取り込みが完了しました。

### 処理結果の確認方法
1. `BeforeAftter.ipynb` を使用して件数の変化を確認
2. 取り込まれたデータの品質をチェック
3. 必要に応じてマートテーブルを再作成
